# Feature Selection

In this notebook, we investigate which features are most informative for predicting the target class.
We combine correlation analysis, tree‑based feature importance, statistical tests, and recursive feature elimination to identify a compact subset of variables that balances predictive power and model simplicity.


In [51]:
import pandas as pd
import seaborn as sns
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
import warnings
warnings.filterwarnings('ignore')

In [52]:
training_data_wt = pd.read_csv('Nata_Files/processed/train_dataset.csv', index_col=0)
testing_data_wt = pd.read_csv('Nata_Files/processed/test_dataset.csv', index_col=0)
prediction_data = pd.read_csv('Nata_Files/processed/pred_scaled.csv', index_col=0)
pastel_d_nata_lrn = pd.read_csv('Nata_Files/learn.csv', index_col=0)
pastel_d_nata_lrn = pastel_d_nata_lrn[pastel_d_nata_lrn['quality_class'].isna()==False]

In [53]:
train_data = training_data_wt.drop('target', axis=1)
train_target = training_data_wt[['target']]
test_data = testing_data_wt.drop('target', axis=1)
test_target = testing_data_wt[['target']]

### Heatmap of correlations between all features and target

<p float="left">
  <img src="Nata_Files/imgs/heatmap-full-combo.png" width="36%" />
  <img src="Nata_Files/imgs/heatmaps-target.png" width="56%" />
</p>

The first heatmap shows pairwise correlations among all numeric features, including the derived `target`.
A very strong positive correlation emerges between `oven_temp`, `egg_temp`, and `final_temp`, suggesting that these three temperature‑related variables are largely redundant and that some can be removed without losing much information.

In the second heatmap, which focuses on correlations with the `target`, `egg_yolk_cnt` stands out as most strongly associated with the outcome, followed by variables such as `baking_duration` and `sugar_content`.
In contrast, the dummy‑encoded `origin` appears only weakly correlated with the target, hinting that bakery location may play a minor role in predicting quality.


### Decision Tree feature importance

To complement the correlation analysis, we fit a Decision Tree Classifier on the original learning dataset (after splitting into training and validation sets with the same proportions used elsewhere).
Using the tree’s `feature_importances_`, we obtain a ranking that reflects how often and how early each feature is used to split the data when predicting `quality_class`.

The resulting importances confirm that `egg_yolk_cnt` and `baking_duration` are particularly influential, with `egg_temp`, `vanilla_extract`, and `sugar_content` also contributing meaningfully.
Meanwhile, `origin_dummy` receives essentially zero importance, reinforcing the earlier indication that origin may be safely dropped for the core predictive model.


In [54]:
X = pastel_d_nata_lrn.drop(['notes_baker','pastry_type', 'quality_class'], axis=1)
y = pastel_d_nata_lrn['quality_class']
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.75, random_state=15)
X_train_dtc = X_train.drop('origin', axis=1)
X_train_dtc['origin_dummy'] = train_data['origin_dummy']

In [55]:
dtc = DecisionTreeClassifier(max_leaf_nodes=112).fit(train_data, train_target)
pd.Series(dtc.feature_importances_, index=train_data.columns, name='feature importance').sort_values(ascending=False)

egg_yolk_cnt         0.332724
baking_duration      0.171064
egg_temp             0.082123
vanilla_extract      0.081416
sugar_content        0.057892
final_temp           0.048887
oven_temp            0.048542
salt_ratio           0.041412
preheat_time         0.036855
cream_fat_content    0.033923
ambient_humidity     0.029941
cooling_period       0.023598
lemon_zest_ph        0.011623
origin_dummy         0.000000
Name: feature importance, dtype: float64

### Comparing approval rates by origin

Before discarding `origin`, we check whether there is a difference in approval rates between Lisbon and Porto.
After normalizing the `origin` labels, we compute the proportion of `OK` natas in each city, revealing that Lisbon bakeries have a noticeably higher approval rate than their counterparts in Porto.


In [56]:
def origin_replace(df):
    origin_update = []
    for city in df['origin']:
        if 'lisboa' in str(city).lower() or 'lisbon' in str(city).lower():
            city= 'Lisbon'
            origin_update.append(city)
        elif 'porto' in str(city).lower():
            city= 'Porto'
            origin_update.append(city)
        else:
            origin_update.append(city)
    df.loc[:,'origin'] = pd.Series(origin_update, index=df.index)
    return df

In [57]:
proportion_testing = origin_replace(X_train)
proportion_testing['quality_class'] = y_train
lisbon = proportion_testing[proportion_testing['origin'] == 'Lisbon']
porto = proportion_testing[proportion_testing['origin'] == 'Porto']
lisbon['quality_class'] = lisbon['quality_class'].map({'OK': 1, 'KO': 0})
porto['quality_class'] = porto['quality_class'].map({'OK': 1, 'KO': 0})
lisbon_percent = 100 / len(lisbon) * lisbon['quality_class'].sum()
porto_percent = 100 / len(porto) * porto['quality_class'].sum()
f'{lisbon_percent:.2f}% of Pastel de Natas from Lisbon got the seal of approval, whereas it was only{porto_percent: .2f}% from Porto.'

'66.06% of Pastel de Natas from Lisbon got the seal of approval, whereas it was only 53.21% from Porto.'

### Chi‑squared test of independence

To formally assess whether `origin` and `quality_class` are statistically associated, we apply a Chi‑squared test of independence.
The test compares the observed contingency table of city by quality with the expected counts under independence; a large p‑value indicates that any differences in approval rates are not strong enough to treat `origin` as a key predictive feature.


In [58]:
def TestIndependence(X,y,var,alpha=0.05):
    dfObserved = pd.crosstab(y,X)
    chi2, p, dof, expected = stats.chi2_contingency(dfObserved.values)
    dfExpected = pd.DataFrame(expected, columns=dfObserved.columns, index = dfObserved.index)
    if p<alpha:
        result="{0} is IMPORTANT for Prediction".format(var)
    else:
        result="{0} is NOT an important predictor. (Discard {0} from model)".format(var)
    print(result)

In [59]:
TestIndependence(X_train['origin'],y_train, 'origin')

origin is IMPORTANT for Prediction


### Final feature subset

Combining the correlation analysis, tree‑based importances, and statistical tests, we refine the feature set used for modeling.
We drop two highly redundant temperature variables (`oven_temp` and `egg_temp`) and also exclude `ambient_humidity`, `preheat_time`, `cream_fat_content`, `lemon_zest_ph`, and `cooling_period`, which showed limited predictive value or added unnecessary complexity.

The remaining features form a more compact and interpretable design matrix that retains the most relevant process and ingredient characteristics for predicting `target`.
We then align the training, testing, and prediction datasets to this reduced feature set and save them for use in the modeling phase.


In [60]:
model = LogisticRegression()
rfe1 = RFE(estimator = model, n_features_to_select = 1)
rfe1.fit_transform(train_data, train_target)
ranking = {col: rfe1.ranking_[i] for i, col in enumerate(train_data.columns)}
pd.Series(ranking, name='temp ranking').sort_values(ascending=True)

egg_yolk_cnt          1
baking_duration       2
final_temp            3
egg_temp              4
sugar_content         5
vanilla_extract       6
oven_temp             7
origin_dummy          8
salt_ratio            9
ambient_humidity     10
cream_fat_content    11
lemon_zest_ph        12
cooling_period       13
preheat_time         14
Name: temp ranking, dtype: int32

In [61]:
cooler_train = train_data.drop(['egg_temp','oven_temp'], axis=1)
rfe1.fit_transform(cooler_train, train_target)
ranking = {col: rfe1.ranking_[i] for i, col in enumerate(cooler_train.columns)}
pd.Series(ranking, name='feature_ranking').sort_values(ascending=True)

egg_yolk_cnt          1
baking_duration       2
final_temp            3
sugar_content         4
vanilla_extract       5
origin_dummy          6
salt_ratio            7
ambient_humidity      8
preheat_time          9
cream_fat_content    10
lemon_zest_ph        11
cooling_period       12
Name: feature_ranking, dtype: int32

In [62]:
rfe7 = RFE(estimator = model, n_features_to_select = 7)
rfe7.fit_transform(cooler_train, train_target)
train_data_lf_nt = cooler_train.loc[:,rfe7.support_]
train_data_lf_nt

,baking_duration,egg_yolk_cnt,final_temp,salt_ratio,sugar_content,vanilla_extract,origin_dummy
id,,,,,,,
3431,-0.763618,1.0,-0.378151,0.166065,-0.263074,-0.388808,1.0
2252,0.657465,1.0,-1.176471,0.928549,0.251981,1.557970,0.0
1049,1.430631,1.5,-1.042017,0.606290,-0.110935,1.428172,0.0
1639,-1.543051,0.0,0.084034,-0.611992,-0.339144,0.025172,1.0
4472,-1.256788,0.5,-0.142857,-1.982852,-0.290016,-1.083501,1.0
...,...,...,...,...,...,...,...
3273,1.556737,0.5,-1.294118,1.111386,-0.164818,1.337343,0.0
2717,-1.256788,0.0,-0.176471,-1.460088,-0.245642,0.386489,1.0
2205,-1.256788,0.0,-0.848739,-0.414064,-0.098257,-0.479032,1.0


In [63]:
train_data_lf = train_data_lf_nt.join(train_target)
test_data_lf = test_data[train_data_lf_nt.columns].join(test_target)
pred_data_lf = prediction_data[train_data_lf_nt.columns].join(train_target)

In [64]:
train_data_lf.to_csv('Nata_Files/less_features/train_data_lf.csv')
test_data_lf.to_csv('Nata_Files/less_features/test_data_lf.csv')
pred_data_lf.to_csv('Nata_Files/less_features/pred_data_lf.csv')